# Export a shapefile of a general MODFLOW model from the NHDPlus dataset

This notebook implements a grid-search approach to finding hydraulic conductivities that result in heads that seem reasonable.

Project specific variables are imported in the model_spec.py and gen_mod_dict.py files that must be included in the notebook directory. The first first includes pathnames to data sources that will be different for each user. The second file includes a dictionary of model-specific information such as  cell size, default hydraulic parameter values, and scenario defintion (e.g. include bedrock, number of layers, etc.). There are examples in the repository. Run the following cells up to the "Run to here" cell to get a pull-down menu of models in the model_dict. Then, without re-running that cell, run all the remaining cells.  Re-running the following cell would re-set the model to the first one in the list, which you probably don't want. If you use the notebook option to run all cells below, it runs the cell you're in, so if you use that option, move to the next cell (below the pull-down menu of models) first.

In [3]:
__author__ = 'Jeff Starn'
%matplotlib notebook
from model_specs import *
from gen_mod_dict import *

import os, sys
import shutil
import numpy as np
import matplotlib.pyplot as plt

from flopy.utils.postprocessing import get_water_table


import flopy.utils.binaryfile as bf
from matplotlib import colors
import flopy as fp
import pandas as pd
# import ipyparallel as ipp
# from model_specs import *
# from gen_mod_dict import *

from ipywidgets import interact, Dropdown
from IPython.display import display

print(sys.version)

3.5.3 |Continuum Analytics, Inc.| (default, Feb 22 2017, 21:28:42) [MSC v.1900 64 bit (AMD64)]


The next cell is a template for making this notebook into a batch script. To do so, save this notebook as a .py file and edit it as follows. Comment out all the notebook-specific commands (drop-down menu stuff and commands preceded by %). Indent everything below the next cell twice so that it falls within the 'for' loop and the 'try' statement. Move the 'except' statement to the end of the script. Comment out lines in the cell after 'Preliminary stuff' so that the model is selected in the 'for' loop from gen_mod_dict. You can leave the print statement in that cell uncommented. 

In [4]:
for key, value in model_dict.items():   # from "gen_mod_dict.py"
    md = key
    ms = model_dict[md]
    print('trying {}'.format(md))
    try:
        pass
    except:
        pass

trying Niantic
trying CoastalCT
trying Assabet


In [5]:
models = list(model_dict.keys())
models.sort()
model_area = Dropdown(
    options=models,
    description='Model:',
    background_color='cyan',
    border_color='black',
    border_width=2)
display(model_area)

### Run to here to initiate notebook

First time using this notebook in this session (before restarting the notebook), run the cells up to this point. Then select your model from the dropdown list above. Move your cursor to this cell and use the toolbar menu Cell --> Run All Below.  After the first time, if you want to run another model, select your model and start running from this cell--you don't need to re-run the cells from the beginning.

## Preliminary stuff

In [79]:
md = model_area.value
ms = model_dict[md]
print('The model being processed is {}\n'.format(md))

The model being processed is CoastalCT



### Get the calibration scenario

Copy model to be calibrated (parent model) to new directory. Define the weight, which is used after all the models are run to find the best set of parameters by weighting the error rates. It is specified here so it can be used in the directory name. Weights > 1 result in fewer dry drains and more cells with heads above land surface. That tends to mean lower hydraulic conductivities.

In [21]:
hydro_wt = 1.0
calib=True

geo_ws = os.path.join(proj_dir, ms['ws'])
parent_ws = os.path.join(geo_ws, scenario_dir)
if calib:
    dir_name = '{}_cal_wt_{:4.2f}'.format(scenario_dir, hydro_wt)
else:
    dir_name = scenario_dir
model_ws = os.path.join(geo_ws, dir_name)

if not os.path.exists(model_ws):
    print('This calibration scenario has not been completed')

Load existing model and some packages needed for parameter estimation

In [89]:
nam_file = '{}.nam'.format(md)
mf = fp.modflow.Modflow.load(nam_file, version='mfnwt', exe_name=mfpth, verbose=False, model_ws=model_ws, load_only=None)

bas = mf.get_package('BAS6')
dis = mf.get_package('DIS')
upw = mf.get_package('UPW')
oc = mf.get_package('OC')
well = mf.get_package('WEL')
head_file_pth = os.path.join(model_ws, '{}.hds'.format(md))

ibound = bas.ibound
botm = dis.getbotm()
hdsobj = bf.HeadFile(head_file_pth)
hds = hdsobj.get_data()
wt = get_water_table(heads=hds, nodata=-8890)

model_file = os.path.join(geo_ws, 'model_grid.csv')
model_grid = pd.read_csv(model_file, na_values=[hnoflo, hdry])

land_surface = model_grid.top
top = land_surface.values.reshape(dis.nrow, dis.ncol)
node = model_grid.node_num.values.reshape(dis.nrow,dis.ncol)

In [90]:
#make a dictionary of heads to export with the model parameters
other_dict={}

#loop through the model layers
for i in range(hds.shape[0]):
    other_dict["head_"+str(i)]=hds[i,:,:]
    
zone_file = os.path.join(model_ws, 'zone_array.npz')
if os.path.isfile(zone_file):
    zones = np.load(zone_file)
    zones = zones['zone']
    for i in range(zones.shape[0]):
        other_dict["surfzone_"+str(i)]=zones[i,:,:]

other_dict["ZoneBud"]=model_grid.zone.values.reshape(dis.nrow, dis.ncol)

other_dict["Pop"]=model_grid.population.values.reshape(dis.nrow, dis.ncol)
other_dict["wt"]=wt
other_dict["DTW_m"]=top - wt
other_dict["land_surface"]=top
other_dict["node"]=node


In [91]:
cbb = fp.utils.CellBudgetFile(os.path.join(model_ws,md+".cbc"))
CGWD = cbb.get_data(text=b' HEAD DEP BOUNDS')[0]
CGWD_DF = pd.DataFrame(CGWD)
CGWD_DF = CGWD_DF.rename(index=str,columns={"node":"node_num"})
#adjust the node numbering
CGWD_DF.node_num = CGWD_DF.node_num-1
model_grid = pd.merge(model_grid,CGWD_DF,"left","node_num", sort=False)
CGWD_q = -1*model_grid.q.values.reshape(dis.nrow, dis.ncol)
other_dict['CGWD_q'] = CGWD_q


#outFile = os.path.join(model_ws,"CGWD_out2.csv")
#np.savetxt(outFile,CGWD,delimiter=',', header=','.join(CGWD.dtype.names), comments="")


In [25]:
#add the combined well flux to the model_grid
#get the well fluxes (layers 3 & 4)
pumpFluxes = well.stress_period_data.df[well.stress_period_data.df.k>1]

#adjust the node numbering
pumpFluxes.node = pumpFluxes.node-1

#aggregate by cell
pumpFluxes = pumpFluxes.groupby('node').sum()
pumpFluxes.reset_index(inplace=True)
model_grid = pd.merge(model_grid,pumpFluxes,"left",left_on="node_num",right_on="node",sort=False)

Well_Flux = model_grid.flux0.values.reshape(dis.nrow,dis.ncol)
other_dict['Well_Flux']=Well_Flux

C:\Users\jbarclay\AppData\Local\Continuum\anaconda3\envs\GenMod\lib\site-packages\pandas\core\generic.py:2773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [26]:
#open the grid specs file to get the xul and yul values
grid_specs_file = os.path.join(geo_ws,"grid_spec.txt")

o_file = open(grid_specs_file)
marker = 0
for line in o_file:
    if marker=="ul":
        coords = line[:-1].split()
        marker=0
    if marker=="rotation":
        rotation = float(line[:-1].split()[1])
        marker=0
    if len(line)>3:
        if line.split()[0]=="Upper" and line.split()[1]=="left":
            marker="ul"
        if line.split()[0]=="Rotation":
            marker = "rotation"
o_file.close()

xul = float(coords[0])
yul = float(coords[1])

In [93]:
#set the spatial reference

mf.sr = fp.utils.reference.SpatialReference(delr=[L]*dis.ncol, delc=[L]*dis.nrow, lenuni=dis.lenuni, xul = xul, yul = yul, rotation = rotation, units="meters", proj4_str="EPSG:5070")

#use the prj file from the domain file
prj = md+"_domain.prj"
prj = os.path.join(geo_ws,prj)
#temporarily change the directory
thisDir = os.getcwd()
os.chdir(model_ws)
fp.export.shapefile_utils.model_attributes_to_shapefile(md+".shp",mf,array_dict=other_dict, prj=prj)

#change the directory back
os.chdir(thisDir)

wrote CoastalCT.shp


In [94]:
model_grid.columns

Index(['node_num', 'ibound', 'stream_order', 'gess_poly', 'lake', 'zone',
       'ned', 'ned_mean', 'catchment', 'soller_thk', 'bedrock_el', 'nlcd',
       'rch_eff_m_Reitz_2013', 'rch_m_Wolock', 'stage', 'segment_len', 'order',
       'reachcode', 'reach_intermit', 'reach_len', 'reach_int', 'ghb',
       'fresh_head', 'top', 'lay', 'row', 'col', 'obs_type', 'obs_type_num',
       'xcr_', 'ycr_', 'xcl_', 'ycl_', 'dist2str', 'q', 'IFACE           '],
      dtype='object')